### NenuFAR Data Rate Calculator

You are in an interactive python notebook. 
- 1) Press the fast-forward button to execute the code
- 2) use the interactive knobs to compute the data rate.



In [12]:
from IPython.display import HTML

In [13]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def KMGT_Bytes(B):
#-----------------------------------------------------------
# B = data volume or rate in Bytes
# XB = data volume or rate in K/M/G/TBytes
# Unit = KB, MB, GB, TB

    Unit='B' ; XB=B
    k=1024.

    if B >= k and B < k**2:
        XB=B/k ; Unit='KB'

    if B >= k**2 and B < k**3:
        XB=B/k**2 ; Unit='MB'

    if B >= k**3 and B < k**4:
        XB=B/k**3 ; Unit='GB'

    if B >= k**4:
        XB=B/k**4 ; Unit='TB'

    return XB,Unit

def data_rate_vol(Nma=96,Nch=64,dt=1,Df=75,Tobs=3600,mode="BF",Sum=False):
    """
    IDL original version v1, PZ, 20190319
    PYTHON transcript v1, JG, 20200509

    Computes and displays NenuFAR data rates and volumes

    INPUTS
    Nma  = number of Mini-Arrays involved
    Nch  = number of channels / 195.3125 kHz subband
    dt   = integration time of spectra / of visibility sets (sec)
    Df   = total bandwidth (MHz)
    Tobs = total observation time (sec)

    KEYWORDS
    mode = "BF"  => Beamformer mode	(uses Nch, dt, Df)
           "IM"  => Imager mode		(uses Nma, Nch, dt, Df)
           "WF"  => Waveform mode	(uses Nma)
    Sum  = 1 line SUMMARY
    
    """
    
    Nsb=min(round(Df/0.1953125),768)

    D_bf_sb=4.*4.*Nch/dt ;  D_bf=D_bf_sb*Nsb ; D_bf_n = D_bf/2 #2 UnDySPuTeD nodes 
    D_im_sb = 8.*4.*Nch*(Nma*(Nma-1)*1./2+Nma)/dt ; D_im = D_im_sb*Nsb ; D_im_n = D_im/4 #4 correlator nodes
    D_wf_sb = 0. ; D_wf = 2.*2.*2e8*Nma ; D_wf_n = D_wf; #1 Waveform node NB0
    
    if mode == "BF":
        D_sb=D_bf_sb ; D_tot=D_bf ; D_n=D_bf_n ; IM=0 ; WF=0
    elif mode == "IM":
        D_sb=D_im_sb ; D_tot=D_im ; D_n=D_im_n ; WF=0
    elif mode == "WF":
        D_sb=D_wf_sb ; D_tot=D_wf ; D_n=D_wf_n
    elif mode == None:
        print("Choose a mode=BF or IM or WF")

    # data rates / sec : /sb, total, /node

    XD_sb,U_sb=KMGT_Bytes(D_sb)
    if not(Sum) and mode!="WF":
        print("Rate=  "+"%8.1f %s/s/SB"%(XD_sb,U_sb)) #,format='(",f8.1,2x,a2,"/s/SB")'

    XD_tot, U_tot=KMGT_Bytes(D_tot) 
    if not(Sum):
        print("Rate=  "+"%8.1f %s/s"%(XD_tot,U_tot)) #,format='("Rate=  ",f8.1,2x,a2,"/s")'

    XD_n, U_n=KMGT_Bytes(D_n)
    if not(Sum):
        print("Rate=  "+"%8.1f %s/s/node"%(XD_n,U_n)) #,format='("Rate=  ",f8.1,2x,a2,"/s/node")
        
        
    # data volumes total for Tobs

    V_tot = D_tot*Tobs
    XV_tot, UV_tot=KMGT_Bytes(V_tot)
    if not(Sum):
        print("Volume= ",'%8.1f  %s'%(XV_tot,UV_tot),"in ",'%7.0f sec'%Tobs)

    if Sum:
        print("%d MA %d ch/SB %7.3f sec %5.1f MHz %6.0f sec :\n Rate = %6.1f %s /s/SB %6.1f %s /s %6.1f %s /s/node :\n Volume = %6.1f %s)"%(Nma,Nch,dt,Df,Tobs, XD_sb,U_sb,XD_tot,U_tot,XD_n,U_n,XV_tot,UV_tot))
  #format='(i3," MA ",i3," ch/SB ",f7.3," sec ",f5.1," MHz ",f6.0," sec : Rate =",f6.1,1x,a2,"/s/SB ",f6.1,1x,a2,"/s ",f6.1,1x,a2,"/s/node : Volume =",f6.1,1x,a2)'

In [14]:
style = {'description_width': 'initial'}

interact(data_rate_vol,Nma=widgets.IntSlider(description="Number of mini-array",min=0,max=96,step=1,value=96,style=style),
         Nch=widgets.IntSlider(description="Number of channels",min=1,max=64,step=1,value=64,style=style), 
         dt=widgets.IntSlider(description="Time integration (s)",min=1,max=60,step=1,value=1,style=style),
        Df=widgets.FloatSlider(description="Observation band (MHz)",min=1.,max=75.,step=0.1953125,value=75,style=style),
         Tobs=widgets.IntSlider(description="Time of observation (s)",min=1,max=36000,step=1,value=3600,style=style),
        mode=[('Imaging', "IM"), ('Beamformer', "BF"),('Waveform', "WF")])

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT05NiwgZGVzY3JpcHRpb249dSdOdW1iZXIgb2YgbWluaS1hcnJheScsIG1heD05Niwgc3R5bGU9U2xpZGVyU3R5bGUoZGVzY3LigKY=


<function __main__.data_rate_vol>